**Nama : Nanang Safiu Ridho**

**Repository GitHub : https://github.com/Nanangsr/Case-Study/tree/main**

**Linkedin  : https://www.linkedin.com/in/nanang-safiu-ridho-804112248/**

GitHub merender file Jupyter Notebook (.ipynb) secara statis saat dilihat di browser, sehingga output visualisasi seperti plot dari Plotly atau Matplotlib sering tidak muncul.

# Setup Database dan Notebook

In [1]:
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv
import plotly.express as px
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

import sys
!{sys.executable} -m pip install "nbformat>=4.2.0"
load_dotenv()

# Setup
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

headers = {
    "apikey": SUPABASE_KEY,
    "Authorization": f"Bearer {SUPABASE_KEY}",
    "Prefer": "count=exact" 
}

def load_table(table_name, batch_size=1000):
    all_data = []
    offset = 0
    
    try:
        while True:
            url = f"{SUPABASE_URL}/rest/v1/{table_name}?select=*&limit={batch_size}&offset={offset}"
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            
            batch = response.json()

            if not batch:
                break
            
            all_data.extend(batch)
            offset += batch_size
            
            print(f"Loading {table_name}: {len(all_data)} rows...", end='\r')
            
            if len(batch) < batch_size:
                break
        
        df = pd.DataFrame(all_data)
        print(f"Tabel '{table_name}' berhasil dimuat: {len(df)} rows                    ")
        return df
        
    except Exception as e:
        print(f"Gagal memuat '{table_name}': {e}")
        return pd.DataFrame()

# Test koneksi
try:
    test_url = f"{SUPABASE_URL}/rest/v1/"
    response = requests.get(test_url, headers=headers)
    print("Koneksi ke Supabase berhasil via REST API!\n")
except Exception as e:
    print(f"Koneksi gagal: {e}")
    exit()

# Load semua tabel
table_names = [
    "employees", "dim_companies", "dim_areas", "dim_positions", 
    "dim_departments", "dim_divisions", "dim_directorates", 
    "dim_grades", "dim_education", "dim_majors",
    "dim_competency_pillars", "performance_yearly", 
    "competencies_yearly", "profiles_psych", "strengths", "papi_scores"
]

dfs = {}
print("Memuat data dari tabel...")
for name in table_names:
    dfs[name] = load_table(name)

print("\nSemua data berhasil dimuat!")
print(f"\nContoh: employees shape = {dfs['employees'].shape}")

# Cek beberapa tabel untuk verifikasi
print("\nRingkasan Data:")
for name in table_names[:5]: 
    print(f"  - {name}: {len(dfs[name])} rows x {len(dfs[name].columns)} columns")

Koneksi ke Supabase berhasil via REST API!

Memuat data dari tabel...
Tabel 'employees' berhasil dimuat: 2010 rows                    
Tabel 'dim_companies' berhasil dimuat: 4 rows                    
Tabel 'dim_areas' berhasil dimuat: 4 rows                    
Tabel 'dim_positions' berhasil dimuat: 6 rows                    
Tabel 'dim_departments' berhasil dimuat: 6 rows                    
Tabel 'dim_divisions' berhasil dimuat: 5 rows                    
Tabel 'dim_directorates' berhasil dimuat: 3 rows                    
Tabel 'dim_grades' berhasil dimuat: 3 rows                    
Tabel 'dim_education' berhasil dimuat: 4 rows                    
Tabel 'dim_majors' berhasil dimuat: 6 rows                    
Tabel 'dim_competency_pillars' berhasil dimuat: 10 rows                    
Tabel 'performance_yearly' berhasil dimuat: 10050 rows                    
Tabel 'competencies_yearly' berhasil dimuat: 100500 rows                    
Tabel 'profiles_psych' berhasil dimuat: 2010 row

# Pembersihan & Persiapan Data

In [2]:
# Gabungkan data demografis dasar dari tabel dimensi ke tabel utama 'employees'
df_main = dfs['employees'].copy()
df_main = pd.merge(df_main, dfs['dim_grades'], on='grade_id', how='left')
df_main = pd.merge(df_main, dfs['dim_education'], on='education_id', how='left')
df_main = pd.merge(df_main, dfs['dim_positions'], on='position_id', how='left')

# Ambil data kinerja tahun terakhir saja agar relevan
df_performance = dfs['performance_yearly'][dfs['performance_yearly']['year'] == dfs['performance_yearly']['year'].max()]
df_main = pd.merge(df_main, df_performance[['employee_id', 'rating']], on='employee_id', how='left')

# Tandai top performer
df_main['is_top_performer'] = df_main['rating'] == 5

df_profiles = dfs['profiles_psych'].copy()
df_profiles['mbti'] = df_profiles['mbti'].str.upper().str.strip()
df_main = pd.merge(df_main, df_profiles, on='employee_id', how='left')

print("Dataset utama siap untuk dianalisis:")
df_main.head()

Dataset utama siap untuk dianalisis:


,employee_id,fullname,nip,company_id,area_id,position_id,department_id,division_id,directorate_id,grade_id,...,rating,is_top_performer,pauli,faxtor,disc,disc_word,mbti,iq,gtq,tiki
0,EMP100000,Rendra Pratama,806137,1,4,1,6,3,3,2,...,3.0,False,86,75,SI,Steadiness-Influencer,None,94.0,33.0,2
1,EMP100001,Wulan Setiawan,476388,4,4,4,5,5,3,1,...,3.0,False,48,52,DS,Dominant-Steadiness,INTP,94.0,17.0,3
2,EMP100002,Julia Jatmiko Situmorang,941921,1,3,5,1,1,3,1,...,4.0,False,66,38,DC,Dominant-Conscientious,None,109.0,20.0,3
3,EMP100003,Oka Halim,751615,1,3,4,2,1,1,2,...,3.0,False,39,63,SI,Steadiness-Influencer,ENTJ,85.0,39.0,3
4,EMP100004,Dwi Pratama,443809,3,4,6,5,3,3,1,...,4.0,False,75,100,None,Steadiness-Conscientious,INTJ,134.0,21.0,9


# Eksplorasi Faktor Kontekstual

## Distribusi Top Performer Berdasarkan Jenjang Jabatan

In [16]:
# Kita mulai dari yang paling dasar: apakah jenjang jabatan (grade) berpengaruh?
fig_grade = px.histogram(df_main,
                         x='name_x',
                         color='is_top_performer',
                         barmode='group',
                         histnorm='percent',
                         title='Distribusi Top Performer Berdasarkan Jenjang Jabatan',
                         labels={'name_x': 'Jenjang Jabatan'})
fig_grade.update_layout(yaxis_title="Persentase dalam Grup (%)")
fig_grade.show()

### Interpretasi

Baik, mari kita mulai investigasi kita pada area pertama yaitu faktor kontekstual. Pertanyaan yang ingin kita jawab adalah, apakah senioritas atau posisi seseorang berpengaruh terhadap kemungkinan mereka menjadi seorang top performer?

Grafik ini membandingkan komposisi top performer di setiap jenjang jabatan. Kita bisa lihat dengan jelas, pada Jenjang Jabatan IV, proporsi top performer (warna oranye) secara signifikan lebih dominan dibandingkan dengan karyawan lainnya. Apa artinya ini bagi kita? Ini bukan sekadar angka. Ini adalah sinyal bahwa Jenjang Jabatan IV kemungkinan adalah sebuah 'posisi kunci' atau 'titik panas' di mana kinerja terbaik paling sering muncul. Apakah ini karena peran di level tersebut memang sangat strategis? Atau apakah proses penempatan talenta kita di level ini sudah sangat efektif?

Jadi, temuan kunci pertama kita adalah: **konteks jabatan itu sangat penting, dan Jenjang Jabatan IV adalah area yang perlu kita pelajari lebih dalam sebagai model kesuksesan.**

## Perbandingan Masa Kerja

In [4]:
# Selanjutnya, bagaimana dengan masa kerja?
fig_yos = px.box(df_main,
                 x='is_top_performer',
                 y='years_of_service_months',
                 title='Perbandingan Masa Kerja (dalam Bulan)',
                 labels={'is_top_performer': 'Grup Karyawan', 'years_of_service_months': 'Masa Kerja (Bulan)'},
                 points='all')
fig_yos.show()

### Interpretasi

Selanjutnya, jika jabatan penting, bagaimana dengan pengalaman yang diukur dari masa kerja?

Grafik box plot ini menunjukkan sebaran masa kerja. Garis tengah yang menunjukkan median top performer yang memiliki nilai 52 jika dibandingkan dengan median grup lainnya dengan nilai 49 memang terlihat tidak terlalu berbeda jauh. Namun kita dapat menggali cerita di baliknya, kita dapat mengatakan bahwa kinerja puncak bukanlah sesuatu yang terjadi dalam semalam. Ada proses pematangan. Karyawan butuh waktu untuk belajar, beradaptasi, dan memberikan dampak maksimal. Menariknya, kita melihat lebih sedikit top performer di rentang masa kerja yang sangat singkat.

Ini memperkuat temuan kita sebelumnya. Temuan kunci kedua adalah: **pengalaman dan retensi karyawan adalah faktor krusial. Kita tidak hanya harus menempatkan orang di posisi yang tepat, tetapi juga memberi mereka waktu untuk bertumbuh menjadi talenta terbaik.**

# Eksplorasi Kinerja

## Distribusi Top Performer di Setiap Departemen

In [5]:
# Ganti nama kolom 'name' di tabel departemen agar unik
dim_departments_renamed = dfs['dim_departments'].rename(columns={'name': 'department_name'})
df_dept = pd.merge(df_main, dim_departments_renamed, on='department_id', how='left')

# Bar chart
fig_dept = px.histogram(df_dept,
                        x='department_name', 
                        color='is_top_performer',
                        barmode='group',
                        histnorm='percent',
                        title='Distribusi Top Performer di Setiap Departemen')
fig_dept.update_layout(yaxis_title="Persentase dalam Grup (%)", xaxis_title="Departemen")
fig_dept.update_xaxes(categoryorder="total descending")
fig_dept.show()

### Interpretasi

Sekarang, mari kita perluas analisis kontekstual kita. Kita tahu jabatan dan pengalaman itu penting, tapi bagaimana dengan lingkungan kerja? Apakah ada departemen tertentu yang lebih subur dalam menumbuhkan talenta terbaik?

Grafik ini menjawab pertanyaan tersebut dengan sangat jelas. Kita bisa melihat bahwa Departemen IT dan Operations secara signifikan memiliki proporsi top performer (batang oranye) yang jauh lebih tinggi dibandingkan dengan proporsi mereka di populasi karyawan umum (batang biru). Sebaliknya, di Departemen HR, situasinya berbalik.

Apa cerita di baliknya?

Ini menunjukkan bahwa **kinerja bukanlah semata-mata hasil dari individu, tetapi juga sangat dipengaruhi oleh lingkungan**. Departemen-departemen yang unggul seperti IT dan Operations kemungkinan besar memiliki beberapa "resep rahasia". Mungkin gaya kepemimpinan di sana lebih efektif dalam mendorong kinerja. Bisa jadi budaya kerjanya lebih kompetitif atau lebih berorientasi pada hasil. Atau, jenis pekerjaan di sana lebih menantang dan memberikan dampak langsung pada bisnis, yang memotivasi karyawan untuk memberikan yang terbaik. Temuan ini sangat penting bagi para pemimpin dan tim HR. Ini menunjukkan bahwa kita bisa belajar dari departemen-departemen "juara" ini untuk mereplikasi praktik-praktik terbaik mereka baik itu dari sisi kepemimpinan, budaya, maupun jenis proyekdi seluruh organisasi.

## Distribusi Top Performer Berdasarkan 10 Jurusan Teratas

In [6]:
# Ganti nama kolom 'name' di tabel jurusan agar unik
dim_majors_renamed = dfs['dim_majors'].rename(columns={'name': 'major_name'})
df_major = pd.merge(df_main, dim_majors_renamed, on='major_id', how='left')

# Ambil 10 jurusan paling umum
top_10_majors = df_major['major_name'].value_counts().head(10).index 
df_major_filtered = df_major[df_major['major_name'].isin(top_10_majors)] 

# Bar chart
fig_major = px.histogram(df_major_filtered,
                         x='major_name', 
                         color='is_top_performer',
                         barmode='group',
                         histnorm='percent',
                         title='Distribusi Top Performer Berdasarkan 10 Jurusan Teratas')
fig_major.update_layout(yaxis_title="Persentase dalam Grup (%)", xaxis_title="Jurusan")
fig_major.update_xaxes(categoryorder="total descending")
fig_major.show()

### Interpretasi

Analisis selanjutnya memperlihatkan apakah ada petunjuk dari latar belakang pendidikan para talenta terbaik kita. Grafik ini menunjukkan komposisi top performer di beberapa jurusan yang paling umum di perusahaan kita. Di sini, kita menemukan pola yang sangat jelas dan bisa langsung ditindaklanjuti. Lihatlah dua jurusan di sebelah kiri, Economics dan Engineering. Proporsi top performer (batang oranye) dari kedua jurusan ini secara signifikan lebih tinggi dibandingkan dengan proporsi mereka di populasi karyawan umum. Sebaliknya, pada jurusan seperti Psychology, trennya justru berbalik.

Apa cerita di balik ini?

Ini bukan berarti jurusan lain tidak baik. Namun, ini adalah bukti kuat bahwa **pola pikir analitis, terstruktur, dan berorientasi bisnis yang biasanya diasah di jurusan Ekonomi dan Teknik, tampaknya menjadi kunci sukses yang sangat efektif di lingkungan kerja kita**. Karyawan dengan latar belakang ini lebih sering mencapai kinerja puncak. Ini adalah wawasan yang sangat berharga untuk strategi rekrutmen kita ke depan. Temuan ini memberikan sinyal yang jelas bahwa kita harus lebih proaktif dalam mencari dan merekrut talenta-talenta terbaik dari fakultas Ekonomi dan Teknik, karena mereka telah terbukti memiliki profil yang paling sesuai untuk menjadi top performer di perusahaan kita.

# Eksplorasi Pilar Kompetensi

## Perbandingan Profil Kompetensi

In [7]:
# Di sini, kita ingin membandingkan profil kompetensi kedua grup secara menyeluruh.

# Menyiapkan data kompetensi terbaru
comp_data = (
    dfs['competencies_yearly']
    .query(f"year == {dfs['competencies_yearly']['year'].max()}")
    .merge(dfs['dim_competency_pillars'], on='pillar_code')
    .merge(df_main[['employee_id', 'is_top_performer']], on='employee_id')
)

# Menghitung rata-rata skor per pilar untuk setiap grup
comp_avg = comp_data.groupby(['pillar_label', 'is_top_performer'])['score'].mean().unstack()

# Radar Chart
fig_comp = go.Figure()

fig_comp.add_trace(go.Scatterpolar(
    r=comp_avg[True],
    theta=comp_avg.index,
    fill='toself',
    name='Top Performer'
))
fig_comp.add_trace(go.Scatterpolar(
    r=comp_avg[False],
    theta=comp_avg.index,
    fill='toself',
    name='Karyawan Lainnya'
))

fig_comp.update_layout(
    title='Perbandingan Profil Kompetensi: Top Performer vs. Lainnya',
    polar=dict(radialaxis=dict(visible=True, range=[1, 5]))
)
fig_comp.show()

### Interpretasi

Baik, setelah kita melihat faktor seperti jabatan, pengalaman, departemen, dan jurusan, sekarang kita masuk ke area yang paling krusial kompetensi. Pertanyaannya, apakah top performer kita memiliki profil keahlian yang berbeda?

Radar chart yang tertampil memetakan profil kompetensi rata-rata dari kedua grup. Area biru merepresentasikan top performer, dan area merah adalah karyawan lainnya. Hal pertama yang langsung terlihat sangat jelas adalah area biru jauh lebih luas. Ini adalah bukti visual yang kuat bahwa **top performer kita unggul secara menyeluruh di semua pilar kompetensi yang kita ukur.**

## Kesenjangan Skor Kompetensi

In [8]:
# Untuk memperjelas di mana letak perbedaan terbesar, kita akan menghitung selisih skor rata-rata antara Top Performer dan Karyawan Lainnya.

# Mengitung selisih skor dan buat kolom baru
comp_avg['selisih_skor'] = comp_avg[True] - comp_avg[False]

# Mengurutkan berdasarkan selisih terbesar agar ceritanya lebih jelas
df_selisih = comp_avg.sort_values(by='selisih_skor', ascending=False).reset_index()

# Bar chart untuk menampilkan selisih tersebut
fig_gap = px.bar(df_selisih,
                 x='pillar_label',
                 y='selisih_skor',
                 title='Kesenjangan Skor Kompetensi (Top Performer vs. Lainnya)',
                 labels={'pillar_label': 'Pilar Kompetensi', 'selisih_skor': 'Selisih Skor Rata-rata'},
                 text_auto='.2f')

fig_gap.update_layout(xaxis_title="", yaxis_title="Kesenjangan Skor")
fig_gap.show()

### Interpretasi

Radar chart sebelumnya sudah memberi kita gambaran umum bahwa top performer unggul di semua lini. Sekarang, mari kita bedah lebih dalam. Grafik di hadapan kita ini secara spesifik mengukur seberapa besar kesenjangan skor antara top performer dan karyawan lainnya, dan hasilnya sangat mencerahkan. Temuan yang paling menonjol dan mungkin sedikit mengejutkan adalah pilar dengan gap tertinggi bukanlah murni soal strategi atau kepemimpinan, melainkan 'Social Empathy & Awareness'.

Apa cerita di balik ini?

Ini menunjukkan bahwa kemampuan untuk "membaca situasi", memahami dinamika tim, dan berempati dengan kebutuhan rekan kerja serta stakeholder adalah pembeda paling fundamental. Diikuti oleh 'Commercial Savvy & Impact' dan 'Value Creation for Users', yang keduanya sangat berorientasi pada hasil bisnis dan kepuasan pelanggan. Jadi, temuan kunci dari analisis ini adalah: **resep rahasia talenta terbaik kita bukanlah sekadar kemampuan teknis, melainkan kombinasi dari kecerdasan sosial, wawasan bisnis, dan fokus pada pelanggan**.

Ini bukan lagi sekadar data, ini adalah peta jalan yang sangat jelas. Jika kita ingin "mencetak" lebih banyak top performer, kita harus memfokuskan investasi, pelatihan, dan program pengembangan kita pada ketiga area krusial ini. Inilah yang benar-benar memisahkan karyawan yang 'baik' dari mereka yang 'luar biasa' di perusahaan kita.

# Eksplorasi Faktor Psikometri

## Perbandingan Skor IQ

In [9]:
# Kita akan cek dua hal utama di sini: IQ (kognitif) dan MBTI (kepribadian).

# Analisis IQ
fig_iq = px.box(df_main.dropna(subset=['iq']), # Hapus baris yg IQ-nya kosong
                x='is_top_performer',
                y='iq',
                color='is_top_performer',
                title='Perbandingan Skor IQ',
                points="all")
fig_iq.show()

### Interpretasi

Sekarang kita masuk ke faktor kognitif, dan di sini kita menemukan hasil yang sangat menarik dan mungkin di luar dugaan. Grafik di hadapan kita membandingkan sebaran skor IQ antara top performer dan karyawan lainnya. Jika kita perhatikan baik-baik, median atau nilai tengah skor IQ antara top performer dan karyawan lainnya **hampir identik**, yaitu di angka 108 dan 109. Jangkauan skor mereka pun sangat mirip.

Apa artinya ini?

Ini adalah temuan yang sangat penting karena membantah asumsi umum bahwa untuk menjadi top performer, seseorang harus memiliki IQ yang jauh lebih superior. Data kita tidak menunjukkan hal tersebut. Ini bukan berarti IQ tidak penting. Sebaliknya, ini menunjukkan bahwa ada sebuah ambang batas atau standar kemampuan kognitif yang dibutuhkan untuk bisa bekerja dengan baik di sini. Namun, setelah standar itu terpenuhi, memiliki IQ yang lebih tinggi tidak secara otomatis membuat seseorang menjadi top performer. Jadi, temuan kunci kita di sini adalah: **kemampuan kognitif adalah 'tiket masuk', bukan faktor penentu kemenangan**. Faktor-faktor lain seperti yang akan kita lihat selanjutnya kemungkinan besar memiliki peran yang jauh lebih besar dalam membedakan kinerja yang baik dari kinerja yang luar biasa.

## Perbandingan MBTI

In [10]:
# Analisis MBTI
fig_mbti = px.histogram(df_main.dropna(subset=['mbti']),
                        x='mbti',
                        color='is_top_performer',
                        barmode='group',
                        histnorm='percent',
                        title='Distribusi Tipe MBTI per Grup')
fig_mbti.update_layout(yaxis_title="Persentase dalam Grup (%)", xaxis={'categoryorder':'total descending'})
fig_mbti.show()

### Interpretasi

Jika bukan IQ yang menjadi pembeda utama, lalu apa? Mari kita lihat profil kepribadian mereka. Grafik ini menunjukkan tipe-tipe MBTI mana yang lebih dominan di masing-masing grup. Di sini, kita melihat pola yang sangat jelas. Tipe-tipe seperti ENFP, ISFP, dan ISTP menunjukkan proporsi yang jauh lebih tinggi di kalangan top performer (batang oranye) dibandingkan dengan populasi karyawan lainnya. Sebaliknya, tipe-tipe yang cenderung lebih introvert dan intuitif seperti INTP dan INTJ justru jauh lebih jarang kita temukan di grup berkinerja puncak.

Apa cerita di balik pola ini?

Ini menunjukkan bahwa ada gaya kerja dan interaksi tertentu yang tampaknya sangat cocok dengan budaya dan tuntutan pekerjaan di perusahaan kita. Dominasi tipe ENFP dan ISFP bisa mengindikasikan bahwa kreativitas, kemampuan beradaptasi, dan kecerdasan emosional dalam berinteraksi dengan orang lain adalah kunci sukses. Kehadiran ISTP juga menyoroti pentingnya pendekatan yang praktis, logis, dan berorientasi pada penyelesaian masalah secara langsung. Jadi, temuan kunci kita di sini adalah: **kesuksesan tidak hanya ditentukan oleh kecerdasan, tetapi juga oleh kecocokan profil kepribadian**. Kita tampaknya lebih menghargai talenta yang adaptif, berorientasi pada tindakan praktis, dan mampu berkolaborasi secara efektif, daripada mereka yang cenderung bekerja secara teoretis dan mandiri.

# Eksplorasi Faktor Perilaku

In [11]:
# Pertanyaan: Apa bakat alami yang menjadi "signature" atau ciri khas dari Top Performer?

# Membuat data 5 bakat teratas dari tiap karyawan.
df_top5_strengths = (
    dfs['strengths']
    .query("rank <= 5")
    .merge(df_main[['employee_id', 'is_top_performer']], on='employee_id')
)

# Menghitung frekuensi 10 bakat teratas KHUSUS untuk Top Performer.
top_performer_strengths = (
    df_top5_strengths.query("is_top_performer == True")['theme']
    .value_counts()
    .head(10)
    .reset_index(name='count')
)

# Menghitung frekuensi 10 bakat teratas KHUSUS untuk Karyawan Lainnya.
other_strengths = (
    df_top5_strengths.query("is_top_performer == False")['theme']
    .value_counts()
    .head(10)
    .reset_index(name='count')
)

# Bar chart 10 Bakat Paling Dominan di Kalangan Top Performer
fig_top = px.bar(top_performer_strengths,
                 x='theme',
                 y='count',
                 title='10 Bakat Paling Dominan di Kalangan Top Performer',
                 labels={'theme': 'Tema Bakat (Strength)', 'count': 'Jumlah Karyawan'},
                 text_auto=True,
                 color_discrete_sequence=['#FF6347'])
fig_top.update_layout(xaxis={'categoryorder':'total descending'})
fig_top.show()

# Bar chart 10 Bakat Paling Umum di Kalangan Karyawan Lainnya
fig_other = px.bar(other_strengths,
                   x='theme',
                   y='count',
                   title='10 Bakat Paling Umum di Kalangan Karyawan Lainnya',
                   labels={'theme': 'Tema Bakat (Strength)', 'count': 'Jumlah Karyawan'},
                   text_auto=True)
fig_other.update_layout(xaxis={'categoryorder':'total descending'})
fig_other.show()

### Interpretasi

Kita sampai pada bagian investigasi bakat alami. Untuk benar-benar memahami apa yang menjadi "mesin pendorong" top performer kita, saya telah memisahkan data mereka. Grafik di hadapan kita menunjukkan DNA kesuksesan di perusahaan kita.

Grafik pertama (oranye) secara spesifik menunjukkan 10 bakat yang paling dominan di kalangan Top Performer. Hasilnya sangat tajam. Kita bisa lihat dengan jelas bahwa 'Futuristic', 'Restorative', dan 'Self-Assurance' adalah tiga bakat yang mendominasi profil mereka. Jika kita bandingkan dengan grafik kedua (biru), yang menunjukkan profil karyawan lainnya, kita melihat cerita yang berbeda. Di sana, bakat seperti 'Positivity', 'Arranger', dan 'Learner' lebih menonjol. Ini adalah bakat-bakat yang sangat baik, namun memiliki fokus yang berbeda.

Apa cerita di balik perbedaan ini?

Ini adalah temuan kita yang paling penting. Ini menunjukkan bahwa talenta terbaik kita memiliki "DNA" atau "signature" bakat yang sangat spesifik, yang berpusat pada dua domain utama:

1. Pemikiran Visioner & Pemecahan Masalah: Bakat seperti 'Futuristic' (visi ke depan) dan 'Restorative' (dorongan alami untuk memperbaiki masalah) menunjukkan bahwa mereka adalah "pemikir" yang proaktif dan solutif.
2. Keyakinan & Eksekusi: Bakat seperti 'Self-Assurance' (keyakinan diri) dan 'Activator' (dorongan untuk memulai tindakan) menunjukkan bahwa mereka tidak hanya berpikir, tetapi juga berani dan mampu untuk bertindak.

Jadi, kesimpulan akhir kita adalah: **Top performer kita adalah individu yang secara alami mampu melihat masa depan, terdorong untuk memperbaiki apa yang rusak, dan memiliki kepercayaan diri untuk mengambil tindakan. Inilah kombinasi bakat yang menjadi pembeda paling kuat, dan ini akan menjadi inti dari Success Formula yang akan kita bangun.**

# Peta Korelasi Antar Variabel

In [12]:
# Menyiapkan data numerik
df_papi_pivot = dfs['papi_scores'].pivot(
    index='employee_id',
    columns='scale_code',
    values='score'
)
df_numeric = df_main[['employee_id', 'rating', 'iq', 'years_of_service_months', 'pauli', 'faxtor']].copy()
df_corr = pd.merge(df_numeric, df_papi_pivot, on='employee_id', how='inner').set_index('employee_id')
correlation_matrix = df_corr.corr()

# Heatmap
fig_corr = px.imshow(correlation_matrix.loc['rating':'rating', :],
                     color_continuous_scale='RdBu_r',
                     title='Heatmap Korelasi: Faktor Apa yang Paling Berhubungan dengan Kinerja?',
                     text_auto='.2f',
                     aspect="auto")
fig_corr.show()

### Interpretasi

Setelah kita melihat berbagai faktor secara terpisah, saya ingin menguji sebuah hipotesis yang umum "Apakah ada satu 'angka ajaib' seperti IQ, masa kerja, atau preferensi kerja tertentu dari tes PAPI yang bisa secara langsung memprediksi kinerja tinggi?"

Untuk menjawabnya, saya membuat heatmap korelasi ini. Warna merah pekat menandakan hubungan positif yang kuat, sementara biru pekat menandakan hubungan negatif yang kuat. Namun, seperti yang bisa kita lihat di sepanjang baris rating kinerja, semua warnanya netral, dan semua angkanya mendekati nol.

Apa artinya ini bagi kita?

Ini adalah salah satu temuan kita yang paling penting: **Tidak ada jalan pintas atau 'angka ajaib' menuju kinerja puncak.**

Data ini membuktikan bahwa kesuksesan di perusahaan kita tidak ditentukan oleh satu faktor numerik tunggal. Seseorang tidak dijamin menjadi top performer hanya karena IQ-nya tinggi, masa kerjanya lama, atau memiliki skor PAPI tertentu. Temuan ini sangat berharga karena mengarahkan kita menjauh dari asumsi yang terlalu sederhana dan memaksa kita untuk melihat gambaran yang lebih kompleks. Ini memperkuat bukti dari analisis kita sebelumnya bahwa kesuksesan adalah hasil dari kombinasi berbagai faktor: kompetensi strategis yang tepat, bakat alami yang khas, dan lingkungan kerja yang mendukung.

# Analisis Interaksi Variabel

In [13]:
# Menyiapkan data
df_sunburst = df_main.dropna(subset=['name_x', 'mbti']).copy()

# Membuat label yang lebih deskriptif untuk legenda
df_sunburst['Status'] = df_sunburst['is_top_performer'].map({True: 'Top Performer', False: 'Karyawan Lainnya'})

# Sunburst Chart
fig_sunburst = px.sunburst(
    df_sunburst,
    path=['Status', 'name_x', 'mbti'],
    title='Peta Komposisi Karyawan: Interaksi Antara Status, Jenjang, dan Tipe MBTI'
)
fig_sunburst.show()

### Interpretasi

Semua analisis kita sebelumnya telah menunjukkan faktor-faktor individual yang penting. Sekarang, mari kita gabungkan semuanya untuk melihat gambaran besarnya. Grafik di hadapan kita ini adalah sebuah sunburst chart, yang bisa kita anggap sebagai "Peta DNA" organisasi kita.

- Lingkaran pusat membagi semua karyawan menjadi dua kelompok: Top Performer (oranye) dan Karyawan Lainnya (biru).
- Lingkaran tengah memecah setiap kelompok berdasarkan jenjang jabatan mereka.
- Lingkaran terluar memecah lagi berdasarkan tipe kepribadian MBTI.

Dengan visualisasi ini, kita bisa menavigasi "DNA" organisasi kita. Jika kita fokus pada segmen 'Top Performer' yang kecil namun sangat penting itu, kita bisa langsung melihat bahwa irisan untuk 'Jenjang Jabatan IV' adalah yang paling besar. Ini mengkonfirmasi temuan kita sebelumnya bahwa level ini adalah "sarang" bagi para talenta terbaik.

#  Faktor Pendorong Utama Kinerja (Success Formula)

In [14]:
# Persiapan Data untuk Model Feature Importance
df_model = df_main.copy()
df_model = pd.merge(df_model, dim_departments_renamed, on='department_id', how='left')
df_model = pd.merge(df_model, dim_majors_renamed, on='major_id', how='left')

# Menambahkan data 5 bakat teratas (seperti sebelumnya, tapi isi nilai kosong dengan 0)
df_top5_strengths = dfs['strengths'].query("rank <= 5")
df_strengths_pivot = pd.crosstab(df_top5_strengths['employee_id'], df_top5_strengths['theme'])
df_model = pd.merge(df_model, df_strengths_pivot, on='employee_id', how='left')
for theme in df_strengths_pivot.columns:
    if theme in df_model.columns:
        df_model[theme].fillna(0, inplace=True)

df_competencies_pivot = dfs['competencies_yearly'].pivot_table(
    index='employee_id', columns='pillar_code', values='score', aggfunc='mean'
)
df_model = pd.merge(df_model, df_competencies_pivot, on='employee_id', how='left')

df_papi_pivot = dfs['papi_scores'].pivot_table(
    index='employee_id', columns='scale_code', values='score', aggfunc='mean'
)
df_model = pd.merge(df_model, df_papi_pivot, on='employee_id', how='left')

# Definisi target dan fitur
target = 'is_top_performer'
df_model.dropna(subset=[target], inplace=True)

# Definisikan semua kolom fitur yang akan digunakan
features = [
    # Kontekstual & Demografis
    'name_x', 'years_of_service_months', 'name_y', 'department_name', 'major_name',
    # Psikometri
    'iq', 'mbti', 'disc',
] + list(df_strengths_pivot.columns) \
  + list(df_competencies_pivot.columns) \
  + list(df_papi_pivot.columns)

features = [f for f in features if f in df_model.columns]

X = df_model[features]
y = df_model[target]

# Preprocessing 
categorical_features = X.select_dtypes(include=['object', 'category']).columns
numerical_features = X.select_dtypes(include=np.number).columns

numerical_transformer = SimpleImputer(strategy='median')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Latih Model dengan Penanganan Imbalance
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))])

model.fit(X, y)

# Ekstrak dan Tampilkan Hasil
importances = model.named_steps['classifier'].feature_importances_
try:
    cat_feature_names = model.named_steps['preprocessor'].named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)
    feature_names = numerical_features.tolist() + list(cat_feature_names)
except AttributeError: # Fallback untuk versi scikit-learn yang lebih lama
    cat_feature_names = model.named_steps['preprocessor'].named_transformers_['cat']['onehot'].get_feature_names(categorical_features)
    feature_names = numerical_features.tolist() + list(cat_feature_names)

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)


# Bar chart untuk menampilkan 10 fitur teratas
fig_importance = px.bar(feature_importance_df.head(10),
                        x='importance',
                        y='feature',
                        orientation='h',
                        title='Peringkat Faktor Pendorong Utama Kinerja Tinggi',
                        text_auto='.3f')
fig_importance.update_layout(yaxis=dict(autorange="reversed"), yaxis_title="")
fig_importance.show()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_23028\1467748489.py:12: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





### Interpretasi

Kita sampai pada puncak dari analisis kita. Setelah melihat setiap faktor secara terpisah, sekarang kita akan menjawab pertanyaan paling penting: "Dari semua faktor ini, mana yang paling berpengaruh dalam menentukan seorang top performer?"

Untuk menjawabnya, saya membangun sebuah model prediktif yang menunjukkan petunjuk atau faktor mana yang paling sering ia gunakan untuk menebak dengan benar. Ini adalah daftar peringkat faktor pendorong kinerja tinggi, diurutkan dari yang paling berpengaruh di atas. Hasilnya sangat jelas dan bisa langsung ditindaklanjuti. Kita bisa lihat bahwa empat dari lima faktor teratas adalah pilar kompetensi, yaitu QDD, STO, SEA, dan LIE.

Apa cerita di balik ini?

Ini adalah DNA kesuksesan di perusahaan kita, yang diungkapkan secara kuantitatif. Temuan ini membuktikan bahwa:

1. Kompetensi adalah Raja: Jauh di atas faktor lain, penguasaan pada pilar-pilar kompetensi spesifik inilah yang menjadi pembeda paling kuat. Ini mengkonfirmasi temuan dari radar chart kita, namun sekarang dengan bobot yang terukur.
2. IQ Adalah Faktor Pendukung Penting: iq muncul di peringkat kelima, menunjukkan bahwa kemampuan kognitif tetap menjadi faktor yang sangat relevan, namun tidak sepenting penguasaan kompetensi kunci.
3. Pengalaman Memiliki Peran: years_of_service_months juga masuk dalam 10 besar, memvalidasi analisis awal kita bahwa pengalaman memang berkontribusi terhadap kinerja puncak.

Ini bukan lagi sekadar korelasi atau perbandingan. Ini adalah peta jalan kuantitatif yang menunjukkan di mana kita harus memfokuskan upaya kita. Peringkat inilah yang akan menjadi dasar paling kuat dan dapat dipertanggungjawabkan untuk menentukan bobot dalam Success Formula kita.

### Success Formula

Dari sini, Success Formula kita dapat dijelaskan melalui beberapa komponen utama, diurutkan berdasarkan tingkat kepentingannya:

1. Kompetensi Inti (Bobot Sangat Tinggi):

- Justifikasi: Analisis Feature Importance secara konsisten menempatkan pilar-pilar kompetensi seperti QDD dan STO di peringkat teratas. Ini adalah pembeda paling kuat. Contoh Variabel: Skor QDD, Skor STO, Skor SEA.

2. Karakteristik Individu (Bobot Tinggi):

- Justifikasi: Faktor psikometri seperti iq dan bakat alami tertentu (strengths) muncul sebagai prediktor kuat setelah kompetensi. Contoh Variabel: Skor iq, Kehadiran bakat 'Futuristic', Kehadiran bakat 'Restorative'.

3. Konteks Organisasional & Pengalaman (Bobot Menengah):

- Justifikasi: Masa kerja (years_of_service_months) dan berada di departemen tertentu (misalnya, IT) secara konsisten menunjukkan pengaruh yang signifikan dalam analisis. Contoh Variabel: years_of_service_months, department_name_IT.